In [3]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [4]:
from ipyleaflet import Map, GeoJSON, Polygon as iPolygon
from django.contrib.gis.geos import Polygon as dPolygon
from rest_framework_gis.serializers import GeoFeatureModelSerializer
from django.contrib.gis.db.models import Union, Extent
from public_data.models import RefPlan
import random

class RefPlanSerializer(GeoFeatureModelSerializer):
    class Meta:
        model = RefPlan
        geo_field = "mpoly"
        fields=('city_insee', 'city_name')
        
class SinglePolySerializer(GeoFeatureModelSerializer):
    class Meta:
        model = RefPlan
        geo_field = "mpoly"
        fields=()

def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
    }

In [5]:


# display the global emprise
qs = RefPlan.objects.all().aggregate(mpoly=Extent("mpoly"))
qs['mpoly']

coords = (qs['mpoly'][1], qs['mpoly'][0], qs['mpoly'][3], qs['mpoly'][2])

poly = dPolygon.from_bbox(coords)

x = (qs['mpoly'][0] + qs['mpoly'][2]) / 2
y = (qs['mpoly'][1] + qs['mpoly'][3]) / 2
(x, y)

m = Map(center=(y, x), zoom=4)
m.add_layer(iPolygon(
    locations=poly.coords,
    color="blue",
    fill_color="blue",
))
m

Map(center=[46.2113160334893, 2.2093133927972097], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [6]:
m = Map(center=(y, x), zoom=5)

qs = RefPlan.objects.all().aggregate(mpoly=Union("mpoly"))
serializer = SinglePolySerializer(qs)


geo_json = GeoJSON(
    data=serializer.data,
    style={'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1},
    hover_style={'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5},
    style_callback=random_color,
)
m.add_layer(geo_json)
m

Map(center=[46.2113160334893, 2.2093133927972097], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [ ]:
class DepSerializer(GeoFeatureModelSerializer):
    class Meta:
        model = RefPlan
        geo_field = "mpoly"
        fields=('dept_name', )

qs = RefPlan.objects.values('dept_name').annotate(mpoly=Union("mpoly")).order_by('dept_name')

serializer = DepSerializer(qs, many=True)

m = Map(center=(y, x), zoom=5)

geo_json = GeoJSON(
    data=serializer.data,
    style={'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1},
    hover_style={'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5},
    style_callback=random_color,
)
m.add_layer(geo_json)
m

In [ ]:
qs = RefPlan.objects.values('region_id', 'region_name')
qs = qs.annotate(mpoly=Union("mpoly")).order_by('region_name')
qs